### GeoPython 2019: Workshop Deep Learning using Airborne Imagery
# Example 1: Classification of Land Use with a Simple Convolutional Neural Network


## Workspace setup
The following steps refer to setting up the workspace in Google CoLab. If you want to run the notebook locally, you can clone the GitHub repository containing the notebook and all data. Please note, that you may need to change some paths according to where you stored the data.

### Download the data
Let's first check if the data is already downloaded, and when not do so

In [0]:
!if ! [ -d /geopython/ ]; then mkdir /geopython; curl -s -L -o /geopython/data.zip https://www.dropbox.com/s/3aogabch1d2sdu8/data.zip?dl=0; unzip -q /geopython/data.zip -d /geopython/; rm /geopython/data.zip; fi

### Install additional module
Then we need to install the missing package pandas_ml

In [0]:
!pip install pandas_ml

### Import of all modules

In [0]:
import os
from os.path import exists, isdir, join, splitext
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas_ml import ConfusionMatrix
from PIL import Image
from tensorflow import keras

pd.set_option('display.max_columns', 25)

### Definition of global constants

In [0]:
TILE_SIZE = 100
BATCH_SIZE = 20
ROOT_PATH = '/geopython/'
DATA_PATH = join(ROOT_PATH, 'data')
PRE_PATH = join(ROOT_PATH, 'models/pretrained.h5')
MODEL_PATH = join(ROOT_PATH, 'models/areal.h5')

## Definition of model architecture
In this example, we will implement a very simple Convolutional Neural Network (CNN). The code is based on the book "Deep Learning with Python" by François Chollet, but adapted to handle aerial imagery instead of cats and dogs.

![Example Architecture](imgs/convnet_architecture.png)

Our network consists of four Convolutional layers, each followed by a MaxPooling layer, and two Fully Connected layers at the end. This simple architecture is chosen, to show the fundamental principles of deep learning with CNNs. For productive use, this network could be exchanged with a much deeper architecture, like ResNet or GoogLeNet. These networks will get you better results, but their training takes more time.

In [0]:
class ConvNet(keras.models.Sequential):
    """
    Definition of a simple CNN architecture based on Keras sequential model
    """
    def __init__(self, img_size, num_classes, dropout=0.5):
        """

        :param img_size: Image size of tiles in pixel
        :param num_classes: Number of classes to distinguish
        :param dropout: Optional definition of dropout rate
                        If 'None' is passed, there will be no dropout layer
        """
        super().__init__()  # Initialize sequential model

        # Definition of the network's layer structure
        # For Conv-layers, the number and size of filter kernels is defined
        self.add(keras.layers.Conv2D(32, (3, 3), activation='relu', 
                                     input_shape=(img_size, img_size, 3)))
        # For pooling-layers, the number of pixels to be pooled is defined
        self.add(keras.layers.MaxPooling2D((2, 2)))
        self.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
        self.add(keras.layers.MaxPooling2D((2, 2)))
        self.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
        self.add(keras.layers.MaxPooling2D((2, 2)))
        self.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
        self.add(keras.layers.MaxPooling2D((2, 2)))

        # Flattening of the resulting feature map to get a 1D-vector
        self.add(keras.layers.Flatten())

        # Add dropout layer, if defined
        if dropout:
            self.add(keras.layers.Dropout(dropout))

        # Fully-Connected layers for classification
        self.add(keras.layers.Dense(512, activation='relu'))
        self.add(keras.layers.Dense(num_classes, activation='softmax'))

    def __str__(self):
        return self.summary()

## Definition of dataset classes

This base class helps us loading the correct data for each class during training, validation and testing. It actually just stores the paths of the respective directories

In [0]:
class Dataset(object):
    """
    Definition of dataset base class
    """

    def __init__(self, directory, classes, names=None):
        """
        Initialize a directory as dataset

        :param directory: Root directory as string
        :param classes: List or tuple containing the names of the subdirectories as strings
        :param names: Optional class names (If not defined, directory names will be used)
        """
        self.base_dir = directory
        self.classes = classes
        self.cls_names = names or classes

    @property
    def train_dir(self):
        return join(self.base_dir, 'train')

    @property
    def validation_dir(self):
        return join(self.base_dir, 'val')

    @property
    def test_dir(self):
        return join(self.base_dir, 'test')

    def summary(self):
        """
        Prints a summary of the dataset
        """
        row_format = '{:>' + str(max(len(max(self.cls_names, key=lambda x: len(x))), 6) + 1) + '}' + '{:>12}' * 3 + '\n'
        text = row_format.format('Class', 'Train', 'Val', 'Test')
        text += '-' * len(text) + '\n'
        for cls, name in zip(self.classes, self.cls_names):
            text += row_format.format(name,
                                      len(listdir(join(self.train_dir, cls))),
                                      len(listdir(join(self.validation_dir, cls))),
                                      len(listdir(join(self.test_dir, cls))))
        print(text)

### Swiss Land Use Dataset ("Arealstatistik")

The swiss land use dataset consists of a grid with a mesh size of 100 meters. Each grid point gets assigned to one of 72 different classes. In this example we will only use the four classes 'Urban', 'Rural', 'Forest' and 'Other', which are actually just aggregated from the detailed labels.

![Example Images](imgs/dataset_examples.jpg)

In [0]:
class ArealStat4(Dataset):
    """
    Predefined class for land use dataset
    """
    def __init__(self, directory):
        super().__init__(directory, ('1', '2', '3', '4'), ('Urban', 'Rural', 'Forest', 'Other'))

## Instantiation of dataset and model

In [0]:
data = ArealStat4(DATA_PATH)
model = ConvNet(TILE_SIZE, len(data.classes))

## Training of the network
To train the network, we first have to choose a loss function and an optimizer.

Then we create data generators that prepare the images for feeding them into the network. These generators load the images from the directories specified in the dataset class.

With the ```fit_generator``` method, we actually start the training process. This is where we can choose the duration of the training (```epochs```).

After training, it's always recommended to save the model for using it later.

In [0]:
# Definition of loss function and optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# Definition of generators for training and validation datasets
train_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
train_data = train_gen.flow_from_directory(data.train_dir,
                                           target_size=(TILE_SIZE, TILE_SIZE),
                                           batch_size=BATCH_SIZE,
                                           class_mode='categorical')

val_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
val_data = val_gen.flow_from_directory(data.validation_dir,
                                       target_size=(TILE_SIZE, TILE_SIZE),
                                       batch_size=BATCH_SIZE,
                                       class_mode='categorical')

# Run training and save history values for plotting later
history = model.fit_generator(train_data,
                              steps_per_epoch=150,
                              epochs=20,
                              validation_data=val_data,
                              validation_steps=25)

# Create snapshot of model weights
model.save(MODEL_PATH)

### Create plots

These line plots give us a sneak peak on how the network performed during the training process. Ideally, the accuracy values are getting bigger and the loss is striving to zero.

In [0]:
epochs = range(1, len(history.history['acc']) + 1)

# Create plot for accuracy values
plt.plot(epochs, history.history['acc'], 'r', label='Training acc')
plt.plot(epochs, history.history['val_acc'], 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

# Create plot for loss values
plt.figure()
plt.plot(epochs, history.history['loss'], 'r', label='Training loss')
plt.plot(epochs, history.history['val_loss'], 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Evaluation of trained model

To evaulate, how good the model has learned to classify land use, we predict the classes of the test dataset. These are images, the network has never seen during the training process.

By comparing the predicted labels with the ground truth, we can calculate key values as precision (which percentage of images classified as 'urban' actually show 'urban') and recall (which percentage of all images showing 'rural' were correctly classified) for each class and for the model in total.

In [0]:
# Load correct model weights
model.load_weights(MODEL_PATH)

ground_truth = []
prediction = []

# Iterate over all images of test dataset
for i, c in enumerate(data.classes):
    for img in [f for f in os.listdir(join(data.test_dir,c)) if f.split('.')[-1] == 'jpg']:
            # Load image and convert to 32-bit float in range [0,1]
            raw = Image.open(join(data.test_dir, c, img))
            raw = raw.resize((TILE_SIZE, TILE_SIZE))
            rgb = raw.convert("RGB")
            arr = np.asarray(rgb, dtype='float32') / 255
            arr = np.expand_dims(arr, axis=0)

            # Feed through network
            out = model.predict(arr, batch_size=1)

            ground_truth.append(i)
            prediction.append(np.argmax(out[0]))

# Calculate confusion matrix
cm = ConfusionMatrix(ground_truth, prediction)
cm.print_stats()

## Inference with new data

When the accuracy of trained model is good enough, we can use it to classify any kind of data. To show this, we load an aerial image with arbitrary size. Then we create tiles with the same size as the patches in our training dataset, which we can feed through our trained network.

### Load image and classify its tiles
We load an image and the pretrained network weights. If you succeeded in training your own weights, you can change it to ```MODEL_PATH``` for using them.

Then we determine into how many tiles our image can be divided and store the first corner for each tile in a list. With this list, we can get the pixel values for each tile and pass them to the network for prediction.

In [0]:
# Definition of file path
OP_PATH = join(ROOT_PATH, 'orthoimage.tiff')

# Load correct model weights
model.load_weights(PRE_PATH)

# Load image
img = Image.open(OP_PATH)
width, height = img.size
vals = np.array(img)

tiles = []
classes = []

# Partition image into tiles with same size as training examples
for i in range(0, width, TILE_SIZE):
    for j in range(0, height, TILE_SIZE):
        if i+TILE_SIZE > width or j+TILE_SIZE > height:
            continue
        tiles.append((i, j))

# Batch-wise classify all tiles
for b in range(0, len(tiles), BATCH_SIZE):
    input_data = np.array([vals[i:i+TILE_SIZE, j:j+TILE_SIZE] for j, i in tiles[b:b+BATCH_SIZE]])
    labels = model.predict(input_data, batch_size=BATCH_SIZE)
    classes += [np.argmax(i) for i in labels]

### Export results to csv file
When classifying a aerial image, we actually want to connect our results to coordinate values. Therefore we parse world file corresponding to our image. This text file contains the transformation parameters for converting pixel coordinates to a national or global coordinate system. Then we can write a csv file, which contatins the classification resuts and the respective coordinates.

#### Worldfile basics
A world file is just a simple text file, containing six parameters for the transformation, each on a separate line.
* x-component of pixel width
* y-component of pixel width
* x-component of pixel height
* y-component of pixel height
* x-coordinate of upper left pixel (center of pixel)
* y-coordinate of upper left pixel (center of pixel)

In the easiest case, the fourth line is the negative value of the first line and the second and third lines are zero. In this case we have square pixels that are parallel to the target coordinate system.

#### Default values
If there is no world file for an image, we set some default values for the coordinate export. The origin of the image is set to 0/0 and the pixel width and height to 1.

In [0]:
DEFAULT_GEO_REF = ((0, 0), (1, 0), (0, -1))
WORLD_FILE_EXTENSIONS = {'.tif': 'tfw', '.tiff': 'tfw', '.png': 'pgw', '.jpg': 'jgw', '.jpeg': 'jgw', '.gif': 'gfw'}


def create_classification_export(tiles, classes, image_path, tile_size=TILE_SIZE):
    """
    Export the classification results to a csv file
    :param tiles: List containing the pixel coordinates of all tile corners
    :param classes: List containing the classification results
    :param image_path: Path to image file
    :param tile_size: Tile size in pixels
    """
    # Determine path to world file from image path
    world_file = '{}.{}'.format(splitext(image_path)[0],
                                WORLD_FILE_EXTENSIONS.get(splitext(image_path)[1], 'undefined'))

    with open('{}_cls.csv'.format(splitext(image_path)[0]), 'w') as fid:
        tiles_world = calculate_tile_centers(tiles, world_file, tile_size)

        for position, cls in zip(tiles_world, classes):
            fid.write('{},{},{}\n'.format(*position, cls))


def calculate_tile_centers(tiles_img, world_file, tile_size=TILE_SIZE):
    """
    Calculate center coordinates for all tiles contained in image
    :param tiles_img: List containing the pixel coordinates of all tile corners
    :param world_file: Path to world file
    :param tile_size: Tile size in pixels
    :return: List containing the coordinates of all tile center points
    """
    if exists(world_file):
        georef = parse_worldfile(world_file)
    else:
        georef = DEFAULT_GEO_REF

    return [(georef[0][0] + georef[1][0]*(x+tile_size/2) + georef[1][1]*(y+tile_size/2),
            georef[0][1] + georef[2][0]*(x+tile_size/2) + georef[2][1]*(y+tile_size/2))
            for x, y in tiles_img]


def parse_worldfile(path):
    """
    Parse a world file
    :param path: Path to world file as string
    :return: Tuple containing the georeferencing values
    """
    with open(path) as fid:
        values = [float(v.strip()) for v in fid.readlines()]

    return ((values[4] - 0.5*values[0] - 0.5*values[2],
            values[5] - 0.5*values[1] - 0.5*values[3]),
            tuple(values[0:2]), tuple(values[2:4]))


#### Actually write csv file

In [0]:
create_classification_export(tiles, [data.cls_names[c] for c in classes], OP_PATH, TILE_SIZE)

### Create overlay image for visualization

For visually analyzing our results, we create an overlay for the aerial image, where every tile gets coloured depending on its classification result. This grid is then alpha-blended onto the actual image so we can see the image and the results combined.

In [0]:
COLORS = ['red', 'yellow', 'green', 'blue']

ovr = Image.new('RGB', img.size, 'white')
for anchor, cls in zip(tiles, classes):
    ovr.paste(Image.new('RGB', (TILE_SIZE, TILE_SIZE), COLORS[cls]), anchor)
out = Image.blend(img, ovr, 0.4)
out.save('_cls'.join(splitext(OP_PATH)))

## Further ressources
* [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python): Beginner-friendly introduction to Deep Learning by François Chollet
* [Awesome Deep Learning](https://github.com/ChristosChristofidis/awesome-deep-learning): Curated list containing great ressources about Deep Learning